In [1]:
import os, time
import h5py
import numpy as npd
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import callbacks
from keras import backend as K

K.set_image_dim_ordering('th')

Using TensorFlow backend.


# Train

In [54]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to c:\users\renato~1.cas\appdata\local\temp\pip-req-build-4926xtj_
  Running setup.py bdist_wheel for keras-vggface: started
  Running setup.py bdist_wheel for keras-vggface: finished with status 'done'
  Stored in directory: C:\Users\RENATO~1.CAS\AppData\Local\Temp\pip-ephem-wheel-cache-vlr7mx2w\wheels\36\07\46\06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
  Found existing installation: Keras-Applications 1.0.4
    Uninstalling Keras-Applications-1.0.4:
      Successfully uninstalled Keras-Applications-1.0.4
  Found existing installation: Keras-Preprocessing 1.0.2
    Uninstalling Keras-Preprocessing-1.0.2:
      Successfully uninstalled Keras-Preprocessing-1.0.2


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.16.0 which is incompatible.
tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.2.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
!pip install keras_vggface

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.16.0 which is incompatible.
tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.2.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# LATINXINAI

# VGG-16 ARCHITECTURE

In [2]:
vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(3,96,96), pooling=None, classes=1000)

In [3]:
vgg16.layers

In [4]:
vgg16.trainable = False

In [71]:
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 96, 96)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 96, 96)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 96, 96)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 48, 48)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 48, 48)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 48, 48)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 24, 24)       0         
__________

In [5]:
from keras.engine import  Model
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# add the model on top of the convolutional base
#custom_vgg_model = Model(vgg16.input,top_model.output)

In [72]:
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1179904   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 1,180,161
Trainable params: 1,180,161
Non-trainable params: 0
_________________________________________________________________


In [6]:
model =  Sequential()
model.add(vgg16)
model.add(top_model)

In [7]:
model.layers

In [65]:
model.load_weights(filepath='weights/latinxinai1_weights.h5')

In [50]:
img_width, img_height = (96,96)
img = load_img('real_test_4113.jpg',target_size=(96,96))
print(img)
imgArray = img_to_array(img)
print(type(imgArray))
imgArray = imgArray.reshape(1,3,img_width, img_height)
imgArray = imgArray/float(255)


<PIL.Image.Image image mode=RGB size=96x96 at 0xE4155C0>
<class 'numpy.ndarray'>


In [51]:
outLabel = int(model.predict_classes(imgArray,verbose=0))
print(outLabel)
# Real: 1    |  Fake: 0

1


In [70]:
import cv2
if __name__ == "__main__":
    cap = cv2.VideoCapture(0)
    img_width = 96
    img_height = 96
    
    cascPath = "haarcascade_frontalface_alt.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)

    sample_number = 1
    count = 0
    measures = np.zeros(sample_number, dtype=np.float)

    while True:
        ret, img_bgr = cap.read()
        if ret is False:
            print("Error not camera found. Please turn it on.")
            break

        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(img_gray, scaleFactor = 1.05, minNeighbors = 5) 

        frame = cv2.resize(img_bgr,(96,96))
        for i, (x, y, w, h) in enumerate(faces):

            imgArray = frame.reshape(1,3,img_width, img_height)
            imgArray = imgArray/float(255)
            if int(model.predict_classes(imgArray,verbose=0)) == 1:
                cv2.rectangle(img_bgr, (x, y), (x + w, y + h), (255, 0, 0), 2)
            else:
                cv2.rectangle(img_bgr, (x, y), (x + w, y + h), (0, 0, 255), 2)
          

        count+=1
        img_bgr = cv2.resize(img_bgr,(480,680))
        cv2.imshow('LATINX', img_bgr)

        key = cv2.waitKey(1)
        if key & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

# RESNET USING TRANSFER LEARNING BASED ON FACENET WEIGHTS

In [107]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to c:\users\renato~1.cas\appdata\local\temp\pip-req-build-fl3e7iy0
  Running setup.py bdist_wheel for keras-vggface: started
  Running setup.py bdist_wheel for keras-vggface: finished with status 'done'
  Stored in directory: C:\Users\RENATO~1.CAS\AppData\Local\Temp\pip-ephem-wheel-cache-k6o1uw90\wheels\36\07\46\06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.16.0 which is incompatible.
tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.2.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [108]:
from keras_vggface.vggface import VGGFace
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import callbacks
import keras

In [110]:
vggface = VGGFace(model='resnet50', input_shape = (3,224,224), pooling = 'avg', include_top = False)

94699520/94694792 [==============================] - 147s 2us/step


C:\Users\renato.castro\AppData\Local\Continuum\anaconda3\envs\opencvkeras\lib\site-packages\keras_vggface\models.py:298: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


In [112]:
from tensorflow.python.keras.models import Sequential 
from tensorflow.python.keras.layers import Dense
from keras.layers import InputLayer

In [113]:
count = 0
for i in vggface.layers:
    count += 1
    if(count == 175):
        print(i)
print(count)

175


In [114]:
import tensorflow as tf
tf.shape(vggface.output)

<tf.Tensor 'Shape:0' shape=(2,) dtype=int32>

In [115]:
vggface.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 3, 224, 224)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 64, 112, 112) 9408        input_2[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 64, 112, 112) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 112, 112) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [120]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, InputLayer
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
#import pandas as pd
#custom parameters
nb_class = 2
hidden_dim = 256

#vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
#last_layer = Input(vggface.output)
#print(last_layer)
#your_input = Input(shape=output_shape_of_resnet)
#x = Flatten(your_input)
#input_shape = vggface.output_shape[1]

last_layer = vggface.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(1, activation='sigmoid', name='classifier')(x)
#vggface.trainable = False

model = Model(vggface.input, out)
count = 0
for layer in model.layers:
    if(count<= 173):
        layer.trainable = False
        count += 1
    else:
        break
        
model.output.trainable = True


layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
#r = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])  

#model = Model(vggface.input, out)

model.compile(loss='binary_crossentropy',
           optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6),
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 3, 224, 224)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 64, 112, 112) 9408        input_2[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 64, 112, 112) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 112, 112) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [121]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
#from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet50 import preprocess_input 
from keras.applications.resnet50 import decode_predictions
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True,
        fill_mode='nearest')

In [ ]:
train_generator = data_generator.flow_from_directory(
            r'../Train',
            target_size = (224,224),
            batch_size = 100,
            class_mode = 'binary')

validation_generator = data_generator.flow_from_directory(
            r'../Validation',
            target_size = (224,224),
            batch_size = 100,
            class_mode = 'binary')

In [123]:
model.load_weights(filepath='weights/latinxinai_resnet_weights.h5')

In [126]:
import numpy as np
from keras.preprocessing import image
img = image.load_img('fake1_test.jpg',target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x) # or version=2
preds = model.predict(x)
print('Correct:', i )
print('Predicted:', np.argmax(preds))


Correct: <keras.layers.pooling.GlobalAveragePooling2D object at 0x0000000023F1A470>
Predicted: 0


In [146]:
import cv2
from keras_vggface import utils
if __name__ == "__main__":
    cap = cv2.VideoCapture(0)
    img_width = 96
    img_height = 96
    
    cascPath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)

    sample_number = 1
    count = 0
    measures = np.zeros(sample_number, dtype=np.float)

    while True:
        ret, img_bgr = cap.read()
        if ret is False:
            print("Camera not found. Turn it on")
            break

        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(img_gray, scaleFactor = 1.05, minNeighbors = 5) 

        frame = cv2.resize(img_bgr,(224,224))
        for i, (x, y, w, h) in enumerate(faces):
            imgArray = frame.reshape(1,3,224, 224)
            imgArray = imgArray/float(255)
            preds = int(model.predict(imgArray,verbose = 0))
            if np.argmax(preds) == 1:
                cv2.rectangle(img_bgr, (x, y), (x + w, y + h), (255, 0, 0), 2)
            else:
                cv2.rectangle(img_bgr, (x, y), (x + w, y + h), (0, 0, 255), 2)
            
        count+=1
        img_bgr = cv2.resize(img_bgr,(480,680))
        cv2.imshow('LATINX', img_bgr)

        key = cv2.waitKey(1)
        if key & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()